# EDSA - Climate Change Belief Analysis 2022
# TeamNM6- 2201ACDS_ NM6

### Notebook created by (June 2022)

- #### Samuel Njoki
- #### Mohamed Abubakar
- #### Ubong Ben
- #### Yinka Akindele
- #### Segun Lawal




# Introduction

## Climate Change Based On Novel Tweet Data




## Problem Statement

Increase Companies Market Research/Advertising Efficiency using Machine Learning to create Marketing tools that can identify whether or not a person believes in climate change and could possibly be converted to a new customer based on their tweets.



<img src="https://i.gifer.com/RD07.gif">

# Table of Contents:
- [Import important libraries and datasets](#section-one)
- [A look at our datasets](#section-two)
- [Data Pre-processing](#section-three)


# Start Comet Experiment
We will be using Comet as version control throughout the development of our model

In [1]:
"""
!pip install comet_ml
from comet_ml import Experiment

# Setting the API key (saved as environment variable)
experiment = Experiment(api_key="THysD8zqvW8wCiFTidV67jLP2",
                        project_name="climate-change-belief-analysis", 
                        workspace="jamakasilwane")                      
"""

'\n!pip install comet_ml\nfrom comet_ml import Experiment\n\n# Setting the API key (saved as environment variable)\nexperiment = Experiment(api_key="THysD8zqvW8wCiFTidV67jLP2",\n                        project_name="climate-change-belief-analysis", \n                        workspace="jamakasilwane")                      \n'

<a id="section-one"></a>
# Importing important libraries and datasets
We first need to load the libraries we are going to use throughout our notebook. Then load our train and test data sets

In [6]:
# Standard libraries
import re
import csv
import nltk
# import spacy
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Style
import matplotlib.style as style 
sns.set(font_scale=1.5)
style.use('seaborn-pastel')
style.use('seaborn-poster')
from PIL import Image
# from wordcloud import WordCloud

# Downloads
# nlp = spacy.load('en')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Preprocessing
# import en_core_web_sm
from collections import Counter
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, wordnet  
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Building classification models
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

[nltk_data] Downloading package punkt to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Segun
[nltk_data]     Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Segun Lawal\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# Import and load dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test_with_no_labels.csv')
sample = pd.read_csv('sample_submission.csv')

<a id="section-two"></a>
# A look at our datasets

In [17]:
# a look at the train and test datasets

print(train.shape)
print(test.shape)

display(train.head())
display(test.head())

percentage_duplicate = round(
    (1 - (train['message'].nunique() / len(train['message']))) * 100, 2)

print("{}% of tweets in train dataset are duplicate".format(
    percentage_duplicate))

(15819, 3)
(10546, 2)


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


10.05% of tweets in train dataset are duplicate


In [16]:
# a look at the unique values in the label column of the train dateframe
train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

The train dataset contains over 15,000 tweets while the test dataset contains over 10,000 tweets.

The tweets are divided into 4 classes:

* [ 2 ] News : Tweets linked to factual news about climate change.

* [ 1 ] Pro : Tweets that support the belief of man-made climate change.

* [ 0 ] Neutral : Tweets that neither support nor refuse beliefs of climate change.

*  [-1 ] Anti : Tweets that do not support the belief of man-made climate change.

10% of the messages in the train dataset were duplicated.

Note: This duplicated messages are a called Retweets 

<a id="section-three"></a>
# Data Pre-processing
We will be preparing (cleaning and organizing) our raw data before having an in-depth Exploratory Data Analysis on the cleansed data. 